## Optimizing Python

#### For any numerical analysis, it is best to use the highly optimized numpy library. 

In [1]:
import numpy as np

### Let's generate a toy dataset of random number vectors

In [2]:
def gen_array(n_vec, n_meas):
    data = list()
    for i in range(0, n_vec):
        data.append(np.random.uniform(size=n_meas))
        #data.append(list(np.random.uniform(size=100)))
    data = np.array(data)
    return(data)

In [3]:
data = gen_array(300, 100)

In [4]:
print(type(data))
print(type(data[0][0]))
print(data.shape)

<class 'numpy.ndarray'>
<class 'numpy.float64'>
(300, 100)


#### Here we define a silly all-by-all distance metric:

In [5]:
def pairwise_distance(X):

    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    D = np.empty((num_vectors, num_vectors), dtype=np.float)
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            d = 0.
            for k in range(num_measurements):
                d += np.subtract(X[i][k], X[j][k])
            D[i, j] = np.sqrt(np.abs(d))
    return(D)

For the 300\*100 matrix, calculating pairwise distance this way means 300\*\*2\*100=9000000 calculations

### Pure Python (and numpy)

In [6]:
%%time
result = pairwise_distance(data)

CPU times: user 23.4 s, sys: 84 ms, total: 23.5 s
Wall time: 23.6 s


In [7]:
print(np.asarray(result))

[[0.         1.36920497 2.04293356 ... 1.81917561 1.35537291 2.36064335]
 [1.36920497 0.         1.51619764 ... 1.1977803  0.19413021 2.72898503]
 [2.04293356 1.51619764 0.         ... 0.92961155 1.5285751  3.12189279]
 ...
 [1.81917561 1.1977803  0.92961155 ... 0.         1.21341015 2.98027464]
 [1.35537291 0.19413021 1.5285751  ... 1.21341015 0.         2.72207141]
 [2.36064335 2.72898503 3.12189279 ... 2.98027464 2.72207141 0.        ]]


### Cython

In [21]:
%load_ext cython

In [22]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

def pairwise_distance_cython(double[:, ::1] X):
    
    cdef int num_vectors = X.shape[0]
    cdef int num_measurements = X.shape[1]
    cdef double d    
    cdef double[:, ::1] D = np.empty((X.shape[0], X.shape[0]), dtype=np.float)
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            d = 0.
            for k in range(num_measurements):
                d += np.subtract(X[i][k], X[j][k])
            D[i, j] = np.sqrt(np.abs(d))
    return(D)

In [23]:
%%time
result = pairwise_distance_cython(data)

CPU times: user 10.4 s, sys: 83.8 ms, total: 10.5 s
Wall time: 10.6 s


In [11]:
print(np.asarray(result))

[[0.         1.36920497 2.04293356 ... 1.81917561 1.35537291 2.36064335]
 [1.36920497 0.         1.51619764 ... 1.1977803  0.19413021 2.72898503]
 [2.04293356 1.51619764 0.         ... 0.92961155 1.5285751  3.12189279]
 ...
 [1.81917561 1.1977803  0.92961155 ... 0.         1.21341015 2.98027464]
 [1.35537291 0.19413021 1.5285751  ... 1.21341015 0.         2.72207141]
 [2.36064335 2.72898503 3.12189279 ... 2.98027464 2.72207141 0.        ]]


### Numba

In [5]:
from numba import jit

In [6]:
@jit
def pairwise_distance_numba(X):

    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    D = np.empty((num_vectors, num_vectors), dtype=np.float)
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            d = 0.
            for k in range(num_measurements):
                d += np.subtract(X[i][k], X[j][k])
            D[i, j] = np.sqrt(np.abs(d))
    return(D)

In [14]:
%%time
result = pairwise_distance_numba(data)

CPU times: user 346 ms, sys: 21.1 ms, total: 367 ms
Wall time: 375 ms


In [15]:
print(np.asarray(result))

[[0.         1.36920497 2.04293356 ... 1.81917561 1.35537291 2.36064335]
 [1.36920497 0.         1.51619764 ... 1.1977803  0.19413021 2.72898503]
 [2.04293356 1.51619764 0.         ... 0.92961155 1.5285751  3.12189279]
 ...
 [1.81917561 1.1977803  0.92961155 ... 0.         1.21341015 2.98027464]
 [1.35537291 0.19413021 1.5285751  ... 1.21341015 0.         2.72207141]
 [2.36064335 2.72898503 3.12189279 ... 2.98027464 2.72207141 0.        ]]


In [16]:
# alternative syntax
import numba
pairwise_distance_numba = numba.jit(pairwise_distance)

In [101]:
%%time
result = pairwise_distance_numba(data)

CPU times: user 9.04 ms, sys: 393 µs, total: 9.43 ms
Wall time: 9.37 ms


In [102]:
print(np.asarray(result))

[[0.         1.34788316 0.37196774 ... 1.72509909 1.17710765 2.29340035]
 [1.34788316 0.         1.39826643 ... 1.07665123 0.65666323 1.85550428]
 [0.37196774 1.39826643 0.         ... 1.76474556 1.23448063 2.32336935]
 ...
 [1.72509909 1.07665123 1.76474556 ... 0.         1.26110446 1.51119763]
 [1.17710765 0.65666323 1.23448063 ... 1.26110446 0.         1.96827405]
 [2.29340035 1.85550428 2.32336935 ... 1.51119763 1.96827405 0.        ]]


We can tell Numba to optimize as much as possible, without using any native Python.

@njit is equivalent to @jit(noPython=True):

In [15]:
from numba import njit

In [29]:
@njit
def pairwise_distance_njit(X):

    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    # numba will throw TypeError if initializing to np.float
    # --> remove this, it will populate to float type on its own
    #D = np.empty((num_vectors, num_vectors), dtype=np.float)
    D = np.empty((num_vectors, num_vectors))
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            d = 0.
            for k in range(num_measurements):
                d += np.subtract(X[i][k], X[j][k])
            D[i, j] = np.sqrt(np.abs(d))
    return(D)

In [17]:
%%time
result = pairwise_distance_njit(data)

CPU times: user 171 ms, sys: 4.31 ms, total: 175 ms
Wall time: 176 ms


In [108]:
print(np.asarray(result))

[[0.         1.34788316 0.37196774 ... 1.72509909 1.17710765 2.29340035]
 [1.34788316 0.         1.39826643 ... 1.07665123 0.65666323 1.85550428]
 [0.37196774 1.39826643 0.         ... 1.76474556 1.23448063 2.32336935]
 ...
 [1.72509909 1.07665123 1.76474556 ... 0.         1.26110446 1.51119763]
 [1.17710765 0.65666323 1.23448063 ... 1.26110446 0.         1.96827405]
 [2.29340035 1.85550428 2.32336935 ... 1.51119763 1.96827405 0.        ]]


In [109]:
print(type(result[0]))

<class 'numpy.ndarray'>


### Comparison with replicates

##### This is an extreme example that favors optimization by numba. In some other cases Cython may perform better.

In [19]:
%timeit -n 1 -r 3 result = pairwise_distance(data)

25.2 s ± 211 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [24]:
%timeit -n 1 -r 3 result = pairwise_distance_cython(data)

12.4 s ± 1.83 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [28]:
%timeit -n 1 -r 100 result = pairwise_distance_numba(data)

8.5 ms ± 781 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [27]:
%timeit -n 1 -r 100 result = pairwise_distance_njit(data)

8.53 ms ± 597 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)


### Multiprocessing (and refactoring)

We can take advantage of multiple cores using the multiprocessing module. In this approach, separate __processes__ are used, __not threads__. The use of threads is generally blocked by Python because of the "Global Interpreter Lock". This was a necessary design feature as a trade-off for the enormous flexibility in memory management that Python makes possible. This means that there is no shared memory when using multiprocessing, and thus the individual tasks must be independent.

Multiprocessing generally works well with lists, where one maps a function to each element of the list and these operations are computed as separated processes on separate cores per element of the list. To do this, we'll need to refactor our silly distance function. One approach would be to populate a list containing each of the vector pairs. The drawback here is the memory overhead of this list object:

In [63]:
# using tuples is about 10X faster for this example
def pairwise_list(X):

    list_of_tuples = list()
    
    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            list_of_tuples.append((X[i],X[j]))
            
    return(list_of_tuples)

In [64]:
list_of_tuples = pairwise_list(data)
print(type(list_of_tuples))
print(len(list_of_tuples))

<class 'list'>
90000


Now we'll need to refactor our function for computing distances:

In [38]:
def pairwise_distance_rf(X):
    
    assert(len(X) == 2)
    assert(len(X[0]) == len(X[1]))
    
    d = 0.
    
    for i in range(len(X[0])):
        d += abs(np.subtract(X[0][i], X[1][i]))
    
    return(d)

In [114]:
%%time
result = list(map(pairwise_distance_rf, list_of_tuples))

CPU times: user 22.3 s, sys: 53 ms, total: 22.3 s
Wall time: 22.4 s


In [33]:
import multiprocessing as mp

In [27]:
pool = mp.Pool(1)

In [28]:
%%time
result = pool.map(pairwise_distance_rf, list_of_tuples)

CPU times: user 1.22 s, sys: 363 ms, total: 1.58 s
Wall time: 24 s


In [116]:
pool.close()

In [30]:
pool = mp.Pool(2)

In [31]:
%%time
result = pool.map(pairwise_distance_rf, list_of_tuples)

CPU times: user 1.49 s, sys: 398 ms, total: 1.89 s
Wall time: 12.9 s


In [32]:
pool.close()

We can of course combine these approaches. Let's make a bigger dataset to really put this to the test:

In [82]:
bigger_data = gen_array(500, 100)

In [83]:
list_of_tuples = pairwise_list(bigger_data)
print(type(list_of_tuples))
print(len(list_of_tuples))

<class 'list'>
250000


This means 250000\*100=25000000 calculations

In [71]:
@jit
def pairwise_distance_rf_numba(X):
    
    assert(len(X) == 2)
    assert(len(X[0]) == len(X[1]))
    
    d = 0.
    
    for i in range(len(X[0])):
        d += np.abs(np.subtract(X[0][i], X[1][i]))
    
    return(d)

In [72]:
@njit
def pairwise_distance_rf_njit(X):
    
    assert(len(X) == 2)
    assert(len(X[0]) == len(X[1]))
    
    d = 0.
    
    for i in range(len(X[0])):
        d += np.abs(np.subtract(X[0][i], X[1][i]))
    
    return(d)

In [93]:
pool = mp.Pool(1)

In [94]:
%%time
result = pool.map(pairwise_distance_rf_numba, list_of_tuples)

CPU times: user 3.35 s, sys: 1.12 s, total: 4.47 s
Wall time: 21.8 s


In [95]:
pool.close()

In [96]:
pool = mp.Pool(1)

In [97]:
%%time
result = pool.map(pairwise_distance_rf_njit, list_of_tuples)

CPU times: user 3.3 s, sys: 1.1 s, total: 4.4 s
Wall time: 19.1 s


In [98]:
pool.close()

Again, @njit is not a significant improvement so Numba is able to compile essentially without any native Python.

Let's now distribute our Numba optimized function across multiple cores:

In [108]:
pool = mp.Pool(2)

In [109]:
%%time
result = pool.map(pairwise_distance_rf_njit, list_of_tuples)

CPU times: user 3.3 s, sys: 979 ms, total: 4.28 s
Wall time: 9.81 s


In [110]:
pool.close()

Compare without numba:

In [42]:
pool = mp.Pool(2)

In [43]:
%%time
result = pool.map(pairwise_distance_rf, list_of_tuples)

CPU times: user 4.29 s, sys: 1.24 s, total: 5.53 s
Wall time: 39.8 s


In [44]:
pool.close()

A significant improvement with Numba again, but clearly Numba really shines when working with nested loops.